<a href="https://colab.research.google.com/github/Vaanya3/indian-stocks-wyckoff-screener/blob/main/wyckoff.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# Wyckoff Analysis for Indian Stocks using NSE Bhavcopy
# Created for Google Colab - Ready to Run!

# =============================================================================
# INSTALLATION AND IMPORTS
# =============================================================================

# Install required packages
!pip install plotly pandas numpy zipfile36 requests beautifulsoup4 yfinance

import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import zipfile
import io
import requests
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

print("✅ All packages installed successfully!")
print("🎯 Wyckoff Analysis Tool for Indian Stocks is ready!")

# =============================================================================
# WYCKOFF ANALYSIS CORE LOGIC
# =============================================================================

class WyckoffAnalyzer:
    def __init__(self, data):
        """
        Initialize Wyckoff Analyzer with OHLCV data
        Data should have columns: Open, High, Low, Close, Volume
        """
        self.data = data.copy()
        self.prepare_data()

    def prepare_data(self):
        """Prepare data for Wyckoff analysis"""
        df = self.data

        # Calculate basic indicators
        df['Range'] = df['High'] - df['Low']
        df['Body'] = abs(df['Close'] - df['Open'])
        df['Upper_Shadow'] = df['High'] - df[['Open', 'Close']].max(axis=1)
        df['Lower_Shadow'] = df[['Open', 'Close']].min(axis=1) - df['Low']

        # Volume analysis
        df['Volume_MA'] = df['Volume'].rolling(window=20).mean()
        df['Volume_Ratio'] = df['Volume'] / df['Volume_MA']

        # Price movement analysis
        df['Price_Change'] = df['Close'].pct_change()
        df['Price_Change_MA'] = df['Price_Change'].rolling(window=10).mean()

        # Effort vs Result (Core Wyckoff principle)
        df['Effort'] = df['Volume'] * df['Range']  # Volume * Price Range
        df['Result'] = abs(df['Close'] - df['Open'])  # Actual price movement
        df['Effort_Result_Ratio'] = df['Effort'] / (df['Result'] + 0.001)  # Avoid division by zero

        # Wyckoff phases identification
        self.identify_wyckoff_phases(df)

        self.data = df

    def identify_wyckoff_phases(self, df):
        """Identify Wyckoff market phases"""
        # Calculate moving averages for trend identification
        df['MA_20'] = df['Close'].rolling(window=20).mean()
        df['MA_50'] = df['Close'].rolling(window=50).mean()

        # Volume surge detection
        df['Volume_Surge'] = df['Volume'] > (df['Volume_MA'] * 1.5)

        # Price action analysis
        df['High_Volume_Low_Movement'] = (df['Volume_Ratio'] > 1.2) & (abs(df['Price_Change']) < 0.02)
        df['Low_Volume_High_Movement'] = (df['Volume_Ratio'] < 0.8) & (abs(df['Price_Change']) > 0.03)

        # Wyckoff phase signals
        phases = []
        for i in range(len(df)):
            if i < 50:  # Need enough data for analysis
                phases.append('Insufficient Data')
                continue

            # Get recent data
            recent = df.iloc[i-20:i+1]

            # Accumulation phase indicators
            if (recent['High_Volume_Low_Movement'].sum() >= 3 and
                recent['Close'].iloc[-1] <= recent['MA_20'].iloc[-1] and
                recent['Volume_Ratio'].mean() > 1.0):
                phases.append('Accumulation')

            # Markup phase indicators
            elif (recent['Close'].iloc[-1] > recent['MA_20'].iloc[-1] and
                  recent['Close'].iloc[-1] > recent['MA_50'].iloc[-1] and
                  recent['Price_Change'].mean() > 0):
                phases.append('Markup')

            # Distribution phase indicators
            elif (recent['High_Volume_Low_Movement'].sum() >= 3 and
                  recent['Close'].iloc[-1] >= recent['MA_20'].iloc[-1] and
                  recent['Volume_Ratio'].mean() > 1.0):
                phases.append('Distribution')

            # Markdown phase indicators
            elif (recent['Close'].iloc[-1] < recent['MA_20'].iloc[-1] and
                  recent['Close'].iloc[-1] < recent['MA_50'].iloc[-1] and
                  recent['Price_Change'].mean() < 0):
                phases.append('Markdown')

            else:
                phases.append('Neutral')

        df['Wyckoff_Phase'] = phases

    def get_current_phase(self):
        """Get the current Wyckoff phase"""
        return self.data['Wyckoff_Phase'].iloc[-1]

    def get_phase_summary(self):
        """Get summary of phases in the data"""
        phase_counts = self.data['Wyckoff_Phase'].value_counts()
        return phase_counts

    def create_wyckoff_chart(self, symbol):
        """Create interactive Wyckoff analysis chart"""
        df = self.data

        # Create subplots
        fig = make_subplots(
            rows=4, cols=1,
            shared_xaxis=True,
            vertical_spacing=0.02,
            subplot_titles=(f'{symbol} - Price Action', 'Volume Analysis', 'Effort vs Result', 'Wyckoff Phases'),
            row_heights=[0.4, 0.2, 0.2, 0.2]
        )

        # Price candlestick chart
        fig.add_trace(
            go.Candlestick(
                x=df.index,
                open=df['Open'],
                high=df['High'],
                low=df['Low'],
                close=df['Close'],
                name='Price'
            ),
            row=1, col=1
        )

        # Moving averages
        fig.add_trace(
            go.Scatter(x=df.index, y=df['MA_20'], name='MA 20', line=dict(color='orange')),
            row=1, col=1
        )
        fig.add_trace(
            go.Scatter(x=df.index, y=df['MA_50'], name='MA 50', line=dict(color='purple')),
            row=1, col=1
        )

        # Volume chart
        colors = ['red' if close < open else 'green'
                 for close, open in zip(df['Close'], df['Open'])]

        fig.add_trace(
            go.Bar(x=df.index, y=df['Volume'], name='Volume', marker_color=colors),
            row=2, col=1
        )

        # Volume MA
        fig.add_trace(
            go.Scatter(x=df.index, y=df['Volume_MA'], name='Volume MA', line=dict(color='blue')),
            row=2, col=1
        )

        # Effort vs Result
        fig.add_trace(
            go.Scatter(x=df.index, y=df['Effort_Result_Ratio'], name='Effort/Result Ratio',
                      line=dict(color='red')),
            row=3, col=1
        )

        # Add horizontal line for effort/result ratio
        fig.add_hline(y=df['Effort_Result_Ratio'].median(), line_dash="dash",
                      line_color="gray", row=3, col=1)

        # Wyckoff phases
        phase_colors = {
            'Accumulation': 'green',
            'Markup': 'blue',
            'Distribution': 'orange',
            'Markdown': 'red',
            'Neutral': 'gray',
            'Insufficient Data': 'lightgray'
        }

        for phase in df['Wyckoff_Phase'].unique():
            phase_data = df[df['Wyckoff_Phase'] == phase]
            fig.add_trace(
                go.Scatter(
                    x=phase_data.index,
                    y=[1] * len(phase_data),
                    mode='markers',
                    name=f'Phase: {phase}',
                    marker=dict(color=phase_colors.get(phase, 'gray'), size=8)
                ),
                row=4, col=1
            )

        # Update layout
        fig.update_layout(
            title=f'Wyckoff Analysis - {symbol}',
            height=800,
            showlegend=True,
            xaxis_rangeslider_visible=False
        )

        # Update y-axis titles
        fig.update_yaxes(title_text="Price", row=1, col=1)
        fig.update_yaxes(title_text="Volume", row=2, col=1)
        fig.update_yaxes(title_text="Effort/Result", row=3, col=1)
        fig.update_yaxes(title_text="Phases", row=4, col=1)

        return fig

# =============================================================================
# NSE BHAVCOPY DATA PROCESSOR
# =============================================================================

class BhavcopyProcessor:
    @staticmethod
    def process_bhavcopy_data(df, symbol):
        """
        Convert NSE Bhavcopy data to OHLCV format for a specific symbol
        Expected Bhavcopy columns: SYMBOL, OPEN, HIGH, LOW, CLOSE, TOTTRDQTY, etc.
        """
        # Common column mapping for NSE Bhavcopy
        column_mapping = {
            'SYMBOL': 'Symbol',
            'OPEN': 'Open',
            'HIGH': 'High',
            'LOW': 'Low',
            'CLOSE': 'Close',
            'LAST': 'Close',  # Fallback for CLOSE
            'TOTTRDQTY': 'Volume',
            'TOTTRDVAL': 'Turnover'
        }

        # Rename columns to standard format
        df_renamed = df.rename(columns=column_mapping)

        # Filter for the specific symbol
        symbol_data = df_renamed[df_renamed['Symbol'] == symbol.upper()].copy()

        if symbol_data.empty:
            available_symbols = df_renamed['Symbol'].unique()[:20]  # Show first 20 symbols
            raise ValueError(f"Symbol '{symbol}' not found in data. Available symbols include: {list(available_symbols)}")

        # Ensure we have the required OHLCV columns
        required_columns = ['Open', 'High', 'Low', 'Close', 'Volume']
        missing_columns = [col for col in required_columns if col not in symbol_data.columns]

        if missing_columns:
            raise ValueError(f"Missing required columns: {missing_columns}")

        # Select and clean the data
        ohlcv_data = symbol_data[required_columns].copy()

        # Convert to numeric
        for col in required_columns:
            ohlcv_data[col] = pd.to_numeric(ohlcv_data[col], errors='coerce')

        # Remove any rows with NaN values
        ohlcv_data = ohlcv_data.dropna()

        if ohlcv_data.empty:
            raise ValueError("No valid data found after cleaning")

        return ohlcv_data

    @staticmethod
    def get_available_symbols(df):
        """Get list of available symbols in the bhavcopy data"""
        if 'SYMBOL' in df.columns:
            return sorted(df['SYMBOL'].unique())
        elif 'Symbol' in df.columns:
            return sorted(df['Symbol'].unique())
        else:
            return []

# =============================================================================
# MAIN INTERFACE FUNCTIONS
# =============================================================================

def upload_and_analyze():
    """Main function to upload bhavcopy and perform Wyckoff analysis"""
    print("🚀 WYCKOFF ANALYSIS FOR INDIAN STOCKS")
    print("=" * 50)
    print("\n📋 Instructions:")
    print("1. Upload your NSE Bhavcopy CSV file using the file upload below")
    print("2. Enter the stock symbol you want to analyze")
    print("3. Get your Wyckoff analysis results!")
    print("\n" + "=" * 50)

def analyze_stock(df, symbol):
    """Analyze a specific stock using Wyckoff method"""
    try:
        print(f"\n🔍 Analyzing {symbol.upper()}...")

        # Process the bhavcopy data
        processor = BhavcopyProcessor()
        ohlcv_data = processor.process_bhavcopy_data(df, symbol)

        print(f"✅ Data processed successfully! Found {len(ohlcv_data)} data points.")

        # Perform Wyckoff analysis
        analyzer = WyckoffAnalyzer(ohlcv_data)

        # Get current phase
        current_phase = analyzer.get_current_phase()

        # Get phase summary
        phase_summary = analyzer.get_phase_summary()

        # Display results
        print(f"\n📊 WYCKOFF ANALYSIS RESULTS FOR {symbol.upper()}")
        print("=" * 50)
        print(f"🎯 Current Wyckoff Phase: {current_phase}")
        print(f"📈 Data Range: {len(ohlcv_data)} trading sessions")

        print(f"\n📋 Phase Distribution:")
        for phase, count in phase_summary.items():
            percentage = (count / len(ohlcv_data)) * 100
            print(f"   {phase}: {count} sessions ({percentage:.1f}%)")

        # Create and display the chart
        print(f"\n🎨 Generating interactive Wyckoff chart for {symbol.upper()}...")
        chart = analyzer.create_wyckoff_chart(symbol.upper())
        chart.show()

        # Provide trading insights
        print(f"\n💡 WYCKOFF TRADING INSIGHTS FOR {symbol.upper()}:")
        print("=" * 50)

        if current_phase == 'Accumulation':
            print("🟢 ACCUMULATION PHASE DETECTED")
            print("   • Smart money is likely accumulating")
            print("   • Look for signs of markup phase beginning")
            print("   • Consider accumulating on weakness")

        elif current_phase == 'Markup':
            print("🔵 MARKUP PHASE DETECTED")
            print("   • Price in uptrend with smart money support")
            print("   • Good time to hold existing positions")
            print("   • Watch for distribution phase signals")

        elif current_phase == 'Distribution':
            print("🟠 DISTRIBUTION PHASE DETECTED")
            print("   • Smart money may be distributing")
            print("   • Prepare for potential markdown phase")
            print("   • Consider reducing positions")

        elif current_phase == 'Markdown':
            print("🔴 MARKDOWN PHASE DETECTED")
            print("   • Price in downtrend")
            print("   • Wait for accumulation phase signals")
            print("   • Avoid new long positions")

        else:
            print("⚪ NEUTRAL/TRANSITION PHASE")
            print("   • Market in transition")
            print("   • Wait for clearer signals")
            print("   • Monitor for phase change")

        print(f"\n⚠️  DISCLAIMER: This analysis is for educational purposes only.")
        print("   Always do your own research before making investment decisions.")

        return analyzer

    except Exception as e:
        print(f"❌ Error analyzing {symbol}: {str(e)}")
        return None

# =============================================================================
# DEMO FUNCTION WITH SAMPLE DATA
# =============================================================================

def create_demo_data():
    """Create demo data for testing the Wyckoff analyzer"""
    dates = pd.date_range(start='2024-01-01', end='2024-12-31', freq='D')
    dates = [d for d in dates if d.weekday() < 5]  # Only weekdays

    # Create realistic stock price data
    np.random.seed(42)
    base_price = 100
    prices = [base_price]
    volumes = []

    for i in range(1, len(dates)):
        # Create some trend and volatility
        trend = 0.001
        volatility = 0.02

        # Add some Wyckoff-like patterns
        if i % 50 == 0:  # Create accumulation/distribution patterns
            volume_mult = np.random.uniform(1.5, 3.0)
            price_change = np.random.uniform(-0.01, 0.01)
        else:
            volume_mult = np.random.uniform(0.5, 1.5)
            price_change = np.random.normal(trend, volatility)

        new_price = prices[-1] * (1 + price_change)
        prices.append(max(new_price, 1))  # Keep price positive

        base_volume = 100000
        volume = int(base_volume * volume_mult)
        volumes.append(volume)

    # Create OHLCV data
    demo_data = []
    for i, date in enumerate(dates[:-1]):  # -1 because we need price changes
        open_price = prices[i]
        close_price = prices[i + 1]

        high_price = max(open_price, close_price) * np.random.uniform(1.00, 1.02)
        low_price = min(open_price, close_price) * np.random.uniform(0.98, 1.00)

        demo_data.append({
            'SYMBOL': 'DEMO',
            'OPEN': round(open_price, 2),
            'HIGH': round(high_price, 2),
            'LOW': round(low_price, 2),
            'CLOSE': round(close_price, 2),
            'TOTTRDQTY': volumes[i]
        })

    return pd.DataFrame(demo_data)

def run_demo():
    """Run a demo analysis with sample data"""
    print("🎮 RUNNING DEMO WITH SAMPLE DATA")
    print("=" * 50)

    demo_df = create_demo_data()
    print(f"✅ Created demo dataset with {len(demo_df)} trading sessions")

    # Analyze the demo stock
    analyzer = analyze_stock(demo_df, 'DEMO')

    print("\n🎯 Demo completed! You can now upload your own NSE Bhavcopy data.")
    return demo_df

# =============================================================================
# FILE UPLOAD AND PROCESSING
# =============================================================================

def process_uploaded_file(file_path):
    """Process uploaded bhavcopy file"""
    try:
        # Try to read as CSV
        if file_path.endswith('.csv'):
            df = pd.read_csv(file_path)
        elif file_path.endswith('.zip'):
            # Handle zip files (common for NSE bhavcopy)
            with zipfile.ZipFile(file_path, 'r') as zip_ref:
                csv_files = [f for f in zip_ref.namelist() if f.endswith('.csv')]
                if not csv_files:
                    raise ValueError("No CSV files found in the zip archive")

                # Read the first CSV file
                with zip_ref.open(csv_files[0]) as csv_file:
                    df = pd.read_csv(csv_file)
        else:
            raise ValueError("Unsupported file format. Please upload CSV or ZIP files.")

        print(f"✅ File processed successfully!")
        print(f"📊 Data shape: {df.shape}")
        print(f"📋 Columns: {list(df.columns)}")

        # Show available symbols
        processor = BhavcopyProcessor()
        symbols = processor.get_available_symbols(df)[:20]  # Show first 20
        print(f"🎯 Sample symbols available: {symbols}")

        return df

    except Exception as e:
        print(f"❌ Error processing file: {str(e)}")
        return None

# =============================================================================
# READY TO USE!
# =============================================================================

print("\n🎉 SETUP COMPLETE!")
print("=" * 50)
print("📚 Available functions:")
print("   • run_demo() - Test with sample data")
print("   • upload_and_analyze() - See instructions")
print("   • process_uploaded_file(file_path) - Process your bhavcopy")
print("   • analyze_stock(df, symbol) - Analyze specific stock")
print("\n🚀 Let's start! Run: run_demo() to see it in action!")

✅ All packages installed successfully!
🎯 Wyckoff Analysis Tool for Indian Stocks is ready!

🎉 SETUP COMPLETE!
📚 Available functions:
   • run_demo() - Test with sample data
   • upload_and_analyze() - See instructions
   • process_uploaded_file(file_path) - Process your bhavcopy
   • analyze_stock(df, symbol) - Analyze specific stock

🚀 Let's start! Run: run_demo() to see it in action!


In [6]:
# Run this in a new cell to test everything works
run_demo()

🎮 RUNNING DEMO WITH SAMPLE DATA
✅ Created demo dataset with 261 trading sessions

🔍 Analyzing DEMO...
✅ Data processed successfully! Found 261 data points.

📊 WYCKOFF ANALYSIS RESULTS FOR DEMO
🎯 Current Wyckoff Phase: Neutral
📈 Data Range: 261 trading sessions

📋 Phase Distribution:
   Markup: 151 sessions (57.9%)
   Insufficient Data: 50 sessions (19.2%)
   Accumulation: 22 sessions (8.4%)
   Neutral: 21 sessions (8.0%)
   Distribution: 9 sessions (3.4%)
   Markdown: 8 sessions (3.1%)

🎨 Generating interactive Wyckoff chart for DEMO...
❌ Error analyzing DEMO: make_subplots() got unexpected keyword argument(s): ['shared_xaxis']

🎯 Demo completed! You can now upload your own NSE Bhavcopy data.


,SYMBOL,OPEN,HIGH,LOW,CLOSE,TOTTRDQTY
0,DEMO,100.00,101.26,96.08,97.88,87454
1,DEMO,97.88,100.32,97.72,98.60,65599
2,DEMO,98.60,99.37,97.17,99.25,55808
3,DEMO,99.25,102.99,98.75,101.35,120807
4,DEMO,101.35,101.73,98.69,100.28,52058
...,...,...,...,...,...,...
256,DEMO,192.59,195.11,188.92,190.63,58870
257,DEMO,190.63,190.96,183.16,185.35,62063
258,DEMO,185.35,186.73,182.09,184.83,86426
259,DEMO,184.83,187.50,182.77,184.64,100341


In [7]:
# Method 1: Upload from computer
from google.colab import files
uploaded = files.upload()

# Get the filename
filename = list(uploaded.keys())[0]
print(f"Uploaded file: {filename}")

# Process the file
df = process_uploaded_file(filename)

Saving sec_bhavdata_full_02062025 - sec_bhavdata_full_02062025.csv.csv to sec_bhavdata_full_02062025 - sec_bhavdata_full_02062025.csv (2).csv
Uploaded file: sec_bhavdata_full_02062025 - sec_bhavdata_full_02062025.csv (2).csv
✅ File processed successfully!
📊 Data shape: (2889, 15)
📋 Columns: ['SYMBOL', 'SERIES', 'DATE', 'PREV_CLOSE', 'OPEN_PRICE', 'HIGH_PRICE', 'LOW_PRICE', 'LAST_PRICE', 'CLOSE', 'AVG_PRICE', 'VOLUME', 'TURNOVER_LACS', 'NO_OF_TRADES', 'DELIV_QTY', 'DELIV_PER']
🎯 Sample symbols available: ['1018GS2026', '20MICRONS', '21STCENMGM', '360ONE', '3IINFOLTD', '3MINDIA', '3PLAND', '563GS2026', '577GS2030', '5PAISA', '63MOONS', '654GS2032', '667GS2035', '667GS2050', '675GS2029', '676GS2061', '679GS2031', '679GS2034A', '68GS2060', '695GS2061']
